In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
! git clone https://github.com/pj35134/diabetes.git

Cloning into 'diabetes'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [ ]:
df = spark.read.csv('diabetes/diabetes.csv',header = True, inferSchema=True)

In [ ]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [ ]:
print((df.count(), len(df.columns)))

(2000, 9)


In [ ]:
from google.colab import files
files.upload()

Saving newfile.csv to newfile (1).csv


{'newfile.csv': b'Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age\n4,187,60,28,0,30.2,0.527,45\n1,50,62,27,50,26.1,0.291,\n2,,64,0,0,23.3,0.672,69\n5,,66,23,90,28.1,0.167,21\n0,54,,20,24,25.3,0.187,25\n'}

In [ ]:
#test_data =spark.read.csv('ff.csv',header=True,inferSchema=True)

In [ ]:
#.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+----+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction| Age|
+-----------+-------+-------------+-------------+-------+----+------------------------+----+
|          4|    187|           60|           28|      0|30.2|                   0.527|  45|
|          1|     50|           62|           27|     50|26.1|                   0.291|null|
|          2|   null|           64|            0|      0|23.3|                   0.672|  69|
|          5|   null|           66|           23|     90|28.1|                   0.167|  21|
|          0|     54|         null|           20|     24|25.3|                   0.187|  25|
+-----------+-------+-------------+-------------+-------+----+------------------------+----+



In [ ]:
from pyspark.sql.functions import *
#df.Glucose.isNull()
for i in test_data.columns:

  print(i+":",df[df[i].isNull()].count())

Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0


In [ ]:
df.columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [ ]:
df.dtypes

[('Pregnancies', 'int'),
 ('Glucose', 'int'),
 ('BloodPressure', 'int'),
 ('SkinThickness', 'int'),
 ('Insulin', 'int'),
 ('BMI', 'double'),
 ('DiabetesPedigreeFunction', 'double'),
 ('Age', 'int'),
 ('Outcome', 'int')]

In [ ]:
df.groupby('Outcome').count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  684|
|      0| 1316|
+-------+-----+



In [ ]:
df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

In [ ]:
print("Total : ", df[df['BloodPressure'] == 0].count())

Total :  90


In [ ]:
def count_zeros():
  

  columns_list = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
  for i in columns_list:

    print(i+":", df[df[i] == 0].count())

In [ ]:
count_zeros()

Glucose: 13
BloodPressure: 90
SkinThickness: 573
Insulin: 956
BMI: 28


In [ ]:
from pyspark.sql.functions import when
from pyspark.sql.functions import mean

for i in df.columns[1:8]:
  print(i)

  data=df.agg({i: 'mean'}).first()[0]
  print(data)
  df = df.withColumn(i, \
             when(df[i] == 0,round(data)).otherwise(df[i]))

Glucose
121.1825
BloodPressure
69.1455
SkinThickness
20.935
Insulin
80.254
BMI
32.192999999999984
DiabetesPedigreeFunction
0.47092999999999974
Age
33.0905


In [ ]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|           69|           21|     80|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


In [ ]:
count_zeros()

Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0


In [ ]:
for i in df.columns:
  print( "Correlation to Outcome for ", i, df.stat.corr('Outcome',i))

Correlation to Outcome for  Pregnancies 0.22443699263363961
Correlation to Outcome for  Glucose 0.48796646527321064
Correlation to Outcome for  BloodPressure 0.17171333286446713
Correlation to Outcome for  SkinThickness 0.17119686818142416
Correlation to Outcome for  Insulin 0.1711763270226193
Correlation to Outcome for  BMI 0.2827927569760082
Correlation to Outcome for  DiabetesPedigreeFunction 0.1554590791569403
Correlation to Outcome for  Age 0.23650924717620253
Correlation to Outcome for  Outcome 1.0


In [ ]:
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'],outputCol='features')
output_data=assembler.transform(df)

In [ ]:
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|
|          0|    145|           69|           21|     80|44.2|                    0.63| 31|      1|[0.0,145.0,69.0,2...|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[0.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          0|    173|           

In [ ]:
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
from pyspark.ml.classification import LogisticRegression
final_data=output_data.select('features','Outcome')
train,test=final_data.randomSplit([0.8,0.3])
models=LogisticRegression(labelCol='Outcome')
model=models.fit(train)

In [ ]:
summary=model.summary


In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions=model.evaluate(test)

In [ ]:
predictions.predictions.show(100)

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,57.0,60.0,21...|      0|[4.12577914779374...|[0.98410579998435...|       0.0|
|[0.0,74.0,52.0,10...|      0|[3.66797552555459...|[0.97510736304010...|       0.0|
|[0.0,74.0,52.0,10...|      0|[3.66797552555459...|[0.97510736304010...|       0.0|
|[0.0,74.0,52.0,10...|      0|[3.66797552555459...|[0.97510736304010...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.50163577006133...|[0.92425641400875...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.50163577006133...|[0.92425641400875...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.71385523237806...|[0.93783927593906...|       0.0|
|[0.0,91.0,68.0,32...|      0|[2.35811822778659...|[0.91357734806211...|       0.0|
|[0.0,93.0,60.0,21...|      0|[2.44008663955625...|[0.91983347684761...|    

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='Outcome')

In [ ]:
evaluator.evaluate(model.transform(test))

0.851191348973607

In [ ]:
from google.colab import files
files.upload()

Saving newfile.csv to newfile.csv


{'newfile.csv': b'Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age\r\n4,187,60,28,0,30.2,0.527,45\r\n1,50,62,27,50,26.1,0.291,28\r\n2,150,64,0,0,23.3,0.672,69\r\n5,82,66,23,90,28.1,0.167,21\r\n0,54,66,20,24,25.3,0.187,25'}

In [ ]:
test_data =spark.read.csv('newfile.csv',header=True,inferSchema=True)

In [ ]:
test_data=assembler.transform(test_data)

In [ ]:
results=model.transform(test_data)

In [ ]:
test_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+--------------------+
|          4|    187|           60|           28|      0|30.2|                   0.527| 45|[4.0,187.0,60.0,2...|
|          1|     50|           62|           27|     50|26.1|                   0.291| 28|[1.0,50.0,62.0,27...|
|          2|    150|           64|            0|      0|23.3|                   0.672| 69|[2.0,150.0,64.0,0...|
|          5|     82|           66|           23|     90|28.1|                   0.167| 21|[5.0,82.0,66.0,23...|
|          0|     54|           66|           20|     24|25.3|                   0.187| 25|[0.0,54.0,66.0,20...|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----

In [ ]:
results.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+--------------------+--------------------+--------------------+----------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|            features|       rawPrediction|         probability|prediction|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+--------------------+--------------------+--------------------+----------+
|          4|    187|           60|           28|      0|30.2|                   0.527| 45|[4.0,187.0,60.0,2...|[-1.7685169944776...|[0.14572685211007...|       1.0|
|          1|     50|           62|           27|     50|26.1|                   0.291| 28|[1.0,50.0,62.0,27...|[4.39056628055707...|[0.98775801468603...|       0.0|
|          2|    150|           64|            0|      0|23.3|                   0.672| 69|[2.0,150.0,64.0,0...|[0.25156893808425...|[0.56256263227529...|       0.0|
|   

In [ ]:
results.select('Age','prediction').show()

+---+----------+
|Age|prediction|
+---+----------+
| 45|       1.0|
| 28|       0.0|
| 69|       0.0|
| 21|       0.0|
| 25|       0.0|
+---+----------+



In [ ]:
model.save("model")

In [ ]:
from pyspark.ml.classification import LogisticRegressionModel
model = LogisticRegressionModel.load('model')
